In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!tar zxvf /content/drive/My\ Drive/Colab\ Notebooks/train.tgz
!pip install wandb

In [ ]:
!wandb login

# CAPTCHA Recognition - 6 digits

In [1]:
from PIL import Image

In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



# Parsing Data

In [2]:
CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

In [19]:
#functions
def parse_label(strIN):
    lb = (strIN==CLASS_NAMES).astype(float)
    return lb

def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
    img = tf.io.decode_jpeg(a)
    img = tf.reshape(img,[60,200,3])
    img = tf.image.adjust_contrast(img, 2)
    img = tf.image.convert_image_dtype(img, dtype=tf.float16)/255.
    img = tf.image.per_image_standardization(img)
    return img

def img_aug(img,lb):
    img = tf.image.adjust_contrast(img, 2)
    img = tfa.image.rotate(img,angles=np.random.choice([-0.07,0.07,-0.05,0.05,-0.09,0.09]), interpolation= 'BILINEAR')
    img = tf.reshape(img,[60,200,3])
    return img,lb

In [20]:
#training data 01
def prepData01():  
  train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
  data = pd.read_csv("./train/data01_train.csv")
  for i in range(len(data)):
      data.iloc[i,1] = list(data.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data)):
          arr[j][i] = data.iloc[i,1][j]
  data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  for i in range(6):
      data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

  train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
  train_data = train_dir.map(lambda x: readimg_to_tensor(x))
  train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
  train_data01=tf.data.Dataset.zip((train_data,train_label))
  validation_split=0.1
  num_elements=50000
  split = int(num_elements*validation_split)
  train_data_gen=iter(train_data01.shuffle(50000,seed = 102).batch(50000))

  img , lb = next(train_data_gen)
  img_train = img[:-split]
  lb_train = (lb[0][:-split],lb[1][:-split],lb[2][:-split],lb[3][:-split],lb[4][:-split],lb[5][:-split])
  img_test = img[-split:]
  lb_test = (lb[0][-split:],lb[1][-split:],lb[2][-split:],lb[3][-split:],lb[4][-split:],lb[5][-split:])
  return img_train, lb_train, img_test, lb_test

In [6]:
#training data 01
def prepData01():  
  train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
  data = pd.read_csv("./train/data01_train.csv")
  for i in range(len(data)):
      data.iloc[i,1] = list(data.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data)):
          arr[j][i] = data.iloc[i,1][j]
  data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  for i in range(6):
      data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))
  

  train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
  train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
  train_data01 = tf.data.Dataset.zip((train_dir,train_label))
  train_data01 = train_data01.shuffle(50000,seed = 102)

  test_data01 = train_data01.skip(45000)
  train_data01 = train_data01.take(45000)

  train_ds = train_data01.map(lambda x,y: (readimg_to_tensor(x),y),num_parallel_calls=tf.data.experimental.AUTOTUNE)
  test_ds = test_data01.map(lambda x,y: (readimg_to_tensor(x),y),num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  train_ds_aug = train_ds.map(lambda x,y:img_aug(x,y),tf.data.experimental.AUTOTUNE)
  train_ds = train_ds.concatenate(train_ds_aug)

  return train_ds , test_ds

In [12]:
for i in range(50000):
    a = tf.io.read_file(trd[i].numpy().decode('utf-8'))
    img = tf.io.decode_jpeg(a)
    img = tf.image.adjust_contrast(img, 2)
    img = tfa.image.rotate(img,angles=np.random.choice([-0.07,0.07,-0.05,0.05,-0.09,0.09]), interpolation= 'BILINEAR')
    img = tf.reshape(img,[60,200,3])
    img = tf.io.encode_jpeg(img, format='rgb', quality=100)
    tf.io.write_file("/train/data02_train_aug/"+str(i)+".jpg", img, name=None)

In [11]:
trd = next(iter(train_dir.batch(50000)))

In [40]:
for i in range(50000):
    img =Image.open("train/data01_train/"+data2.iloc[:,0][i])
    img = img.rotate(np.random.randint(-7,7))
    img = ImageEnhance.Contrast(img).enhance(2)
    img.save("./train/data01_train_aug/"+data2.iloc[:,0][i])

In [22]:
from PIL import ImageEnhance

In [61]:
train_ds,test_ds = prepData01()
train_ds = train_ds.cache()
train_ds =  train_ds.batch(64)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.cache()
test_ds = test_ds.batch(64)
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
#training data 02
def prepData02(): 
  train_dir_data02 = os.path.abspath(os.getcwd())+"/train/data02_train/"
  data2 = pd.read_csv("./train/data02_train.csv")
  for i in range(len(data2)):
      data2.iloc[i,1] = list(data2.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data2)):
          arr[j][i] = data2.iloc[i,1][j]
  data2 = data2.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  #to one hot
  for i in range(6):
      data2.iloc[:,i+1] = data2.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data2.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data2.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data2.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data2.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data2.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data2.iloc[:,6].values))


  train_dir = tf.data.Dataset.list_files(train_dir_data02+'*.jpg',shuffle=False) 
  train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
  train_data01 = tf.data.Dataset.zip((train_dir,train_label))
  train_data01 = train_data01.shuffle(50000,seed = 102)

  test_data01 = train_data01.skip(45000)
  train_data01 = train_data01.take(45000)

  train_ds = train_data01.map(lambda x,y: (readimg_to_tensor(x),y),num_parallel_calls=tf.data.experimental.AUTOTUNE)
  test_ds = test_data01.map(lambda x,y: (readimg_to_tensor(x),y),num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  train_ds_aug = train_ds.map(lambda x,y:img_aug(x,y),tf.data.experimental.AUTOTUNE)
  train_ds = train_ds.concatenate(train_ds_aug)

  return train_ds , test_ds

In [21]:
img_train, lb_train, img_test, lb_test = prepData01()

# Model configeration

In [7]:
import wandb
from tensorflow.keras import layers
from tensorflow.keras import backend

In [8]:
filepath = "h5s/"#"/content/drive/Shared drives/TingWeis_Drive/Colab Notebooks/Saved Model/"

In [9]:
config_default = {
    "weight_regu":1e-4,
    "LR":1e-3,
    "dropout":0.3,
    "name":"Mymodel",
    "batch":256
}

In [13]:
  #railway model add 1 256 layer
def railway_model(config=config_default):
    initializer = tf.keras.initializers.he_normal( seed = 3)
    alpha = config["weight_regu"]  # weight decay coefficient
    regularizer = tf.keras.regularizers.l2(alpha)
    dropout_rate =config["dropout"]
    inn = Input((60, 200, 3))
    out = inn
    out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)
    out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)
    out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)
    out = Conv2D(filters=256, kernel_size=(3, 3), padding='same',activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=256, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Flatten()(out)
    sep = Dropout(dropout_rate)(out)
    dig1 = Dense(36, name='digit1', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    dig2 = Dense(36, name='digit2', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    dig3 = Dense(36, name='digit3', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    dig4 = Dense(36, name='digit4', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    dig5 = Dense(36, name='digit5', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    dig6 = Dense(36, name='digit6', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    model = tf.keras.models.Model(inputs=inn, outputs=[dig1, dig2,dig3,dig4,dig5,dig6],name=config["name"])
    model.compile(
        loss=[
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
        ],
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"], beta_1=0.9),
        metrics=['accuracy'])
    model.summary()
    return model

# Fit model

In [14]:
def trainModel(img_train, lb_train, img_test, lb_test,callB,epochs=100,config = config_default):
  history = model.fit(
        img_train,
        lb_train,
        batch_size = config["batch"],
        shuffle = True,
        validation_data=(img_test, lb_test),
        epochs=epochs,
        verbose=1,
        callbacks = callB
        )

In [15]:
def switch_to_SGD():
  model.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy()
    ],
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), 
    metrics=['accuracy'])

In [16]:
class myCallback(tf.keras.callbacks.Callback):
  def __init__(self, config=config_default,**kwargs):
    self.config = config
    self.best = 0
    self.epochs = 0
    self.wait = 0
    self.reduce_once = False
    self.patient = 10  #wait 8 epochs for early stopping
  def on_epoch_end(self, epoch, logs=None):
    self.epochs+=1
    b= np.prod([logs['digit1_accuracy'],
              logs['digit2_accuracy'],
              logs['digit3_accuracy'],
              logs['digit4_accuracy'],
              logs['digit5_accuracy'],
              logs['digit6_accuracy'],
              ])
    a= np.prod([logs['val_digit1_accuracy'],
              logs['val_digit2_accuracy'],
              logs['val_digit3_accuracy'],
              logs['val_digit4_accuracy'],
              logs['val_digit5_accuracy'],
              logs['val_digit6_accuracy'],
              ])
    wandb.log({"loss":logs["loss"],"val_loss" : logs["val_loss"],"accuracy": b, "val_accuracy":a, "epoch": self.epochs})
    print("\nepoch: {}, loss: {:.4f}, val_loss: {:.4f}, accuracy: {:.5f}, val_accuracy: {:.5f}".format(self.epochs,logs["loss"],logs["val_loss"],b,a))
    if a > self.best:
      print("                            ***** accuracy improved from {:.5f} to {:.5f}!! *****".format(self.best,a))
      print("Model saved to "+filepath+self.config["name"]+".hdf5\n" )
      self.best = a
      self.model.save(filepath+self.config["name"]+".hdf5")
      self.wait = 0
      wandb.run.summary["best_val_accuracy"] = a
      wandb.run.summary["best_epoch"] = self.epochs
    else:
      if self.wait>=self.patient:   #switch to SGD or reduce lr if already on SGD 
        if hasattr(self.model.optimizer,'momentum') and not self.reduce_once:     #reduce lr on SGD if not done it once yet
          self.model.optimizer.lr=self.model.optimizer.lr/2
          print("\n\n Change leraning rate to {}, continued ... \n".format(self.model.optimizer.lr))
          self.wait=0
          self.reduce_once=True
        else:
          self.model.stop_training = True
          self.wait=0
        print("Model is not learning, training stop at {}".format(epoch))
      print("Model accuracy did not improve from {:.4f}\n".format(self.best))
      self.wait+=1
      
class Switch_SGD_callback(tf.keras.callbacks.Callback):
  def __init__(self,on_train_end,config=config_default, **kwargs):
    self.do_on_train_end=on_train_end
    self.config = config
  def on_train_end(self, logs=None):
      self.do_on_train_end(self.config)

def do_after_train(config=config_default):
    print("\n ******** Switching to SGD!! **********")
    switch_to_SGD()
    trainModel(img_train, lb_train, img_test, lb_test,[myCB],config=config)

In [ ]:
#choose model
model = railway_model(config=myconfig)

In [33]:
myconfig = {
    "stack" : "conv 3x3 32,32,64,64,128,128,256,256",# str2 : 64   ,block1 : 64,128,256,512",
    "weight_regu" : 5e-3,
    "dropout" : 0.3,
    #"preact" : False,
    "name" : "railway_aug_d3e-1",
    "output_WR" : True,
    "total_params" : 1727352,
    "opt_schedule" : "1e-3 adam + 1e-4 SGD",
    "batch" : 32,
    "test_aug":True,
    "standardized":True,
    #"GlobalAvg":True,
    #"preact":"include shortcut",
    "LR":1e-3
}

In [37]:
model.optimizer.lr=1e-6

In [38]:

trainModel(img_train, lb_train, img_test, lb_test,[myCB],config = myconfig)

Epoch 1/100
1406/1407 [============================>.] - ETA: 0s - loss: 0.9162 - digit1_loss: 0.0098 - digit2_loss: 0.0073 - digit3_loss: 0.0072 - digit4_loss: 0.0086 - digit5_loss: 0.0112 - digit6_loss: 0.0114 - digit1_accuracy: 0.9980 - digit2_accuracy: 0.9986 - digit3_accuracy: 0.9988 - digit4_accuracy: 0.9983 - digit5_accuracy: 0.9975 - digit6_accuracy: 0.9978Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable

epoch: 121, loss: 0.9162, val_loss: 0.9340, accuracy: 0.98903, val_accuracy: 0.98213
                            ***** accuracy improved from 0.98213 to 0.98213!! *****
Model saved to h5s/railway_aug_d3e-1.hdf5

1407/1407 [==============================] - 103s 73ms/step - loss: 0.9162 - digit1_loss: 0.0098 - digit2_loss: 0.0073 - digit3_loss: 0.0072 - digit4_loss: 0.0086 - digit5_loss: 0.0112 - digit6_loss: 0.0114 - digit1_accuracy: 0.9980 - digit2_accuracy: 0.9986 - digit3_accuracy: 0.9988 - digit4_accuracy: 0.9983 

KeyboardInterrupt: 

In [ ]:
model.save(filepath+myconfig["name"]+"1.hdf5")

In [28]:
#train for 100 epochs, switch to SGD if accuracy does not increase for 10 epochs, abort training it happens again

wandb.init(project="my-project-railway",reinit=True,name=myconfig["name"],config=myconfig) 

#ini call back
myCB = myCallback(config = myconfig)
#choose model
model = railway_model(config=myconfig)
trainModel(img_train, lb_train, img_test, lb_test,[myCB,Switch_SGD_callback(do_after_train,config = myconfig)],config = myconfig)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
Model: "railway_aug_d3e-1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 60, 200, 3)] 0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 60, 200, 32)  896         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 58, 198, 32)  9248        conv2d_24[0][0]                  
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 58, 198, 32)  128         conv2d_

ResourceExhaustedError:  OOM when allocating tensor with shape[256,32,58,198] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node railway_aug_d3e-1/batch_normalization_12/FusedBatchNormV3 (defined at <ipython-input-14-fb6fd848924e>:10) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_611845]

Function call stack:
train_function


In [ ]:
myconfig = {
    "stack" : "conv 3x3 32,32,64,64,128,128,256,256",# str2 : 64   ,block1 : 64,128,256,512",
    "weight_regu" : 5e-3,
    "dropout" : 0.3,
    #"preact" : False,
    "name" : "railway_2_b64_wr5e-3",
    "output_WR" : True,
    "total_params" : 1727352,
    "opt_schedule" : "1e-3 adam + 1e-4 SGD",
    "batch" : 64,
    #"GlobalAvg":True,
    #"preact":"include shortcut",
    "LR":1e-3
}
#train for 100 epochs, switch to SGD if accuracy does not increase for 10 epochs, abort training it happens again

wandb.init(project="my-project-railway",reinit=True,name=myconfig["name"],config=myconfig) 

#ini call back
myCB = myCallback(config = myconfig)
#choose model
model = railway_model(config=myconfig)
trainModel(img_train, lb_train, img_test, lb_test,[myCB,Switch_SGD_callback(do_after_train)],config = myconfig)

In [ ]:
#functions
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
    img = tf.io.decode_jpeg(a)
    img = tf.image.convert_image_dtype(img, dtype=tf.float16)/255.
    #img = tf.image.per_image_standardization(img)
    return img
#choose data
img_train, lb_train, img_test, lb_test= prepData01()

In [ ]:
myconfig = {
    "stack" : "conv 3x3 32,32,64,64,128,128,256,256",# str2 : 64   ,block1 : 64,128,256,512",
    "weight_regu" : 5e-3,
    "dropout" : 0.3,
    #"preact" : False,
    "name" : "railway_2_b256_no_st",
    "output_WR" : True,
    "total_params" : 1727352,
    "opt_schedule" : "1e-3 adam + 1e-4 SGD",
    "batch" : 256,
    "standardization":False,
    #"GlobalAvg":True,
    #"preact":"include shortcut",
    "LR":1e-3
}
#train for 100 epochs, switch to SGD if accuracy does not increase for 10 epochs, abort training it happens again

wandb.init(project="my-project-railway",reinit=True,name=myconfig["name"],config=myconfig) 

#ini call back
myCB = myCallback(config = myconfig)
#choose model
model = railway_model(config=myconfig)
trainModel(img_train, lb_train, img_test, lb_test,[myCB,Switch_SGD_callback(do_after_train)],config = myconfig)

In [ ]:
myconfig = {
    "stack" : "conv 3x3 32,32,64,64,128,128,256,256",# str2 : 64   ,block1 : 64,128,256,512",
    "weight_regu" : 5e-3,
    "dropout" : 0.3,
    #"preact" : False,
    "name" : "railway_2_b64_no_st",
    "output_WR" : True,
    "total_params" : 1727352,
    "opt_schedule" : "1e-3 adam + 1e-4 SGD",
    "batch" : 64,
    "standardization":False,
    #"GlobalAvg":True,
    #"preact":"include shortcut",
    "LR":1e-3
}
#train for 100 epochs, switch to SGD if accuracy does not increase for 10 epochs, abort training it happens again

wandb.init(project="my-project-railway",reinit=True,name=myconfig["name"],config=myconfig) 

#ini call back
myCB = myCallback(config = myconfig)
#choose model
model = railway_model(config=myconfig)
trainModel(img_train, lb_train, img_test, lb_test,[myCB,Switch_SGD_callback(do_after_train)],config = myconfig)

In [ ]:
#functions
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
    img = tf.io.decode_jpeg(a)
    #img = tf.image.convert_image_dtype(img, dtype=tf.float16)/255.
    #img = tf.image.per_image_standardization(img)
    return img
#choose data
img_train, lb_train, img_test, lb_test= prepData01()

In [ ]:
myconfig = {
    "stack" : "conv 3x3 32,32,64,64,128,128,256,256",# str2 : 64   ,block1 : 64,128,256,512",
    "weight_regu" : 5e-3,
    "dropout" : 0.3,
    #"preact" : False,
    "name" : "railway_2_b64",
    "output_WR" : True,
    "total_params" : 1727352,
    "opt_schedule" : "1e-3 adam + 1e-4 SGD",
    "batch" : 64,
    "normalize":False,
    "standardization":False,
    #"GlobalAvg":True,
    #"preact":"include shortcut",
    "LR":1e-3
}
#train for 100 epochs, switch to SGD if accuracy does not increase for 10 epochs, abort training it happens again

wandb.init(project="my-project-railway",reinit=True,name=myconfig["name"],config=myconfig) 

#ini call back
myCB = myCallback(config = myconfig)
#choose model
model = railway_model(config=myconfig)
trainModel(img_train, lb_train, img_test, lb_test,[myCB,Switch_SGD_callback(do_after_train)],config = myconfig)

In [ ]:
#manual switch to SGD
switch_to_SGD()
trainModel(img_train, lb_train, img_test, lb_test,[myCB])

In [ ]:
#Manual save
filepath=""
model.save(filepath)